<a href="https://colab.research.google.com/github/ddekun/Intro_Neural_Networks/blob/lesson4/lesson4/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Домашнее задание №4. Сверточные нейронные сети

In [1]:
from __future__ import print_function
import keras # расскоментируйте эту строку, чтобы начать обучение
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [2]:
# установка параметров нейросети
batch_size = 32
num_classes = 10
epochs = 2
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [3]:
# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [4]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [5]:
# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [7]:
# инициализация RMSprop optimizer
opt = keras.optimizers.Adam()

In [8]:
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [9]:
# data augumentation процедура позволяющая за счет искажений изображений увеличить количество тренировочных данных
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation в реальном времени')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    # запуск data augmentation через fit
    #datagen.fit(x_train)

    # запуск data augmentation через fit_generator
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Использование data augmentation в реальном времени


<ipython-input-9-d6375ca3e678>:38: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train, y_train,


Epoch 1/2
1563/1563 [==============================] - 328s 209ms/step - loss: 1.6435 - accuracy: 0.3949 - val_loss: 1.4029 - val_accuracy: 0.4917
Epoch 2/2
1563/1563 [==============================] - 317s 203ms/step - loss: 1.2987 - accuracy: 0.5324 - val_loss: 1.0597 - val_accuracy: 0.6227


In [10]:
# сохранение модели и весов
# if not os.path.isdir(save_dir):
#    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
# model.save(model_path)
print('сохранить обученную модель как %s ' % model_path)

сохранить обученную модель как /content/saved_models/keras_cifar10_trained_model.h5 


In [11]:
# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 13s 42ms/step - loss: 1.0597 - accuracy: 0.6227
Test loss: 1.0597459077835083
Test accuracy: 0.6226999759674072


**1. Попробовать улучшить точность распознования образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложить анализ с описанием того, что улучшает работу нейронной сети и что ухудшает.**

Итого:
отключение аугментации ухудшает результат. улучшение приносит увеличение количества эпох. еще прирост дала замена оптимизатора на Adam. также пробовал менять дропаут и незначительно число нейронов в слоях, однако какого-либо ощутимого эффекта (по сравнению с влиянием эпох и оптимизатора) не заметил.

**2.Описать также в анализе какие необоходимо внести изменения в получившуюся у вас нейронную сеть если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET**

Датасеты различаются размерами картинок, а значит как минимум необходимо корректировать входной и выходной слои нейросети. Например, в CIFAR100 выделены 100 классов изображений, соответственно на выходе нашей нейросети должно быть 100 нейронов (вместо 10 для cifar10). Аналогично, формат изображений MNIST 28х28х1, что меняет входные парметры (для cifar10 32х32х3). В IMAGENET интересней - как я понял там 14,197,122 картинок и 21841 категория, причем изображения разного размера. Но, возможно, можно их предварительно ресайзить под один размер.